<a href="https://colab.research.google.com/github/Mamolinac/Business-intelligence/blob/main/2_Reto_%C3%81rboles_de_decisi%C3%B3n_Mayda_Molina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Se procede con la carga de las librerías de trabajo

In [ ]:
import numpy as np
import pandas as pd

#Librerías Arbol
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Conectar al drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Se procede con la carga de los datos

In [ ]:
nxl='/content/drive/MyDrive/2. CollegeAdmisions_Int_M.xlsx'
import pandas as pd
XDB=pd.read_excel(nxl, sheet_name=0) #Base AAA
XDB=XDB.dropna() #Eliminar filas vacías
XDB.head(100) #Mostrar las 100 lineas del archivo

#Cambiar de la variable 'Gender' el M o F por 0 y 1
XDB['Gender'] = XDB['Gender'].map({'M': 0, 'F': 1})

#seleccionar las variables
XD=XDB[['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT']]
yd=XDB[['Admitted']]

2. Determinar el número de personas que fueron admitidas en la universidad.

In [ ]:
#Cambiar la variable 'Admitted' el Yes or No por 1 y 0
XDB['Admitted'] = XDB['Admitted'].map({'Yes': 1, 'No': 0})

#Contamos el número de datos de la base de datos
total_datos=len(XDB)
print(f"El número total de datos de la base de datos es: {total_datos}")

#Ahora contamos el número de personas admitidas
admitidos = XDB['Admitted'].sum()
print(f"El número de personas admitidas es: {admitidos}")

El número total de datos de la base de datos es: 1528
El número de personas admitidas es: 1303


3. Se procede con la implementación del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini', max_depth=4)
mar.fit(XD,yd)

#Se procede con la gráfica
from pydotplus import graph_from_dot_data #Me cra la gráfica
from sklearn.tree import export_graphviz  #Me exporta a PNG, TIFF


ve=['Edu_Parent1','Edu_Parent2','Gender','White','Asian','HSGPA','SAT/ACT'] #Nombres para el árbol
dot_data=export_graphviz(mar,feature_names=ve) #Crear el gráfico
graph=graph_from_dot_data(dot_data)
graph.write_png('Reto 2.ArbolDecision_M.png')

True

4. Determinar el número de Nodos puros para el arbol

In [ ]:
import numpy as np

def contar_nodos_puros(arbol_decision, X, y):
    """
    Cuenta el número de nodos puros en un árbol de decisión.

    Args:
        arbol_decision: Un objeto DecisionTreeClassifier entrenado.
        X: Los datos de entrada.
        y: Las etiquetas de salida.

    Returns:
        El número de nodos puros en el árbol.
    """

    n_nodos = arbol_decision.tree_.node_count
    valores_nodos = arbol_decision.tree_.value
    nodos_puros = 0

    for i in range(n_nodos):
        if np.max(valores_nodos[i]) == np.sum(valores_nodos[i]):
            nodos_puros += 1

    return nodos_puros

# Ejemplo de uso:
num_nodos_puros = contar_nodos_puros(mar, XD, yd)
print(f"Número de nodos puros en el árbol: {num_nodos_puros}")

Número de nodos puros en el árbol: 7


5. Llevar acabo pronóstico de la admisión para las personas agrupadas en la base de datos

In [ ]:
# Cargar la base de datos de pronóstico
XDB0 = pd.read_excel(nxl, sheet_name=1)
XDB0.head(100)

# Convertir 'Gender' a valores numéricos
XDB0['Gender'] = XDB0['Gender'].map({'M': 0, 'F': 1})

# Convertir 'Admitted' a valores numéricos
XDB0['Admitted'] = XDB0['Admitted'].map({'Yes': 1, 'No': 0})

# Seleccionar las variables de entrada y salida
XD0 = XDB0[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']].values
YD0 = XDB0[['Admitted']].values  # Esto es solo para referencia, no se usa en la predicción

# Realizar el pronóstico de admisión para todos los estudiantes
y_pred = mar.predict(XD0)

# Crear el informe en Excel con los resultados
df = pd.DataFrame(np.column_stack((XDB0.index, XD0, y_pred)))
df.columns = ['Solicitante', 'Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT', 'Admitted_Pred']

# Guardar el resultado en un archivo de Excel
df.to_excel("/content/drive/MyDrive/CollegeAdmisions_Pronostico.xlsx")

print("El pronóstico de admisión ha sido guardado en 'CollegeAdmisions_Pronostico.xlsx'.")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


El pronóstico de admisión ha sido guardado en 'CollegeAdmisions_Pronostico.xlsx'.


**Descripción de los datos**




En este ejercicio, el análisis se enfocará en siete variables clave que se detallarán más adelante. El objetivo es evaluar la precisión del modelo de árboles de decisión en la estimación de la probabilidad de admisión de los estudiantes a una universidad.

**Descripción de las variables**

**Edu_Parent1:** Nivel educativo alcanzado por el primer padre o tutor del solicitante.

**Edu_Parent2:** Nivel educativo alcanzado por el segundo padre o tutor del solicitante.
Gender: Género del solicitante, donde 0 representa masculino y 1 femenino.

**White:** Variable categórica que indica si el solicitante se identifica como blanco (1) o no (0).
Asian: Variable categórica que señala si el solicitante se identifica como asiático (1) o no (0).

**HSGPA:** Promedio de calificaciones obtenido por el solicitante durante la educación secundaria.

**SAT/ACT:** Puntaje obtenido en los exámenes de admisión estandarizados SAT o ACT.

**Admitted_Pred:** Variable binaria que indica si el solicitante fue admitido (1) o no (0) en la universidad.

4. Evaluar el comportamiento del modelo.


In [ ]:
ydp=mar.predict(XD) #Pronóstico sobre la base de datos original
cm=confusion_matrix(yd,ydp)
print("La matriz de confusión es: \n",cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]


#Métricas de desempeño
EX=(VP+VN)/(VP+VN+FP+FN) #Exactitud admitiendo o no admitiendo
print('La exactitud del modelo es:\n',EX)

TE=(FN+FP)/(VP+VN+FP+FN) #Tasa de error: Es la proporción de predicciones incorrectas sobre el total de predicciones. Indica el porcentaje de estudiantes que el modelo clasificó erróneamente.
print('La tasa de error del modelo es:\n',TE)

SE=VP/(VP+FN) #Sensibilidad correctamente admitiendo
print('La sensibilidad del modelo es:\n',SE)

SP=VN/(VN+FP) #Especificidad correctamente no admitiendo
print('La especificidad del modelo es:\n',SP)

PR=VP/(VP+FP) #Precisión de los admitidos los correctos
print('La precisión del modelo es:\n',PR)

PRNeng=VN/(VN+FN) #Precisión Negativa  de los no admitidos correctos
print('La precisión negativa del modelo es:\n',PRNeng)


La matriz de confusión es: 
 [[  20  205]
 [   2 1301]]
La exactitud del modelo es:
 0.8645287958115183
La tasa de error del modelo es:
 0.1354712041884817
La sensibilidad del modelo es:
 0.9984650805832693
La especificidad del modelo es:
 0.08888888888888889
La precisión del modelo es:
 0.8638778220451527
La precisión negativa del modelo es:
 0.9090909090909091


**Análisis de las métricas**


El modelo de clasificación fue evaluado utilizando un total de 1528 registros, donde 1303 estudiantes fueron admitidos y 225 no lo fueron.

De acuerdo con la matriz de confusión, el modelo identificó correctamente a 1301 estudiantes admitidos y a 20 no admitidos. No obstante, cometió 205 errores al clasificar como falsos positivos y solo tuvo 2 casos en los que admitidos fueron clasificados erróneamente como no admitidos falsos negativos.

La exactitud general del modelo es del 86.45%, lo que indica un buen desempeño en la clasificación de los estudiantes admitidos o no que identifica correctamente el modelo.  La sensibilidad alcanza un 99.85%, lo que significa que el modelo es altamente efectivo para identificar a los estudiantes admitidos. Además, la precisión del 86.39% sugiere que la mayoría de los casos predichos como admitidos realmente lo son. Sin embargo, la especificidad es baja, con un 8.89%, lo que implica que el modelo tiene dificultades para reconocer correctamente a los estudiantes no admitidos.
La precisión negativa del 90.91% indica que la mayoría de los casos identificados como no admitidos fueron correctamente clasificados.

En resumen, el modelo es confiable para predecir la admisión de estudiantes, pero no tanto para aquellos que no fueron admitidos.

